<img src="http://www.integraconference.info/wp-content/uploads/2017/08/Vendor-Logo-AmerisourceBergen.png" width=60%>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
from pandas.api.types import is_numeric_dtype
from scipy.stats import ttest_ind
from sklearn.feature_selection import RFE
# from sklearn.linear_model import LogisticRegression
# import statsmodels.formula.api as sm

import warnings
warnings.filterwarnings('once')

In [4]:
%matplotlib inline
# rcParams['figure.figsize'] = 6, 5
# sb.set_('whitegrid')
sb.set(font='Roboto', style='whitegrid')

In [5]:
dtype_dic= {'GCNSEQ_NBR':str,'NDC':str}
data = pd.read_csv('dataset_big.csv', header=0, dtype=dtype_dic)

FileNotFoundError: [Errno 2] File b'dataset_big.csv' does not exist: b'dataset_big.csv'

In [ ]:
data.info()

In [ ]:
# Set up basic functions
# Excluding outliers: +- 3 SDEV
# def removeOutliers(data,field):
#     result = data[np.abs(data[field]-data[field].mean()) <= (3*data[field].std())]
#     return result

In [ ]:
data1 = data[data['PURCH_QTY_NDC_26WKS_MTRC']>0].reset_index().reset_index()
data1 = data1.drop(columns=['index', 'ID'])
data1.rename(columns={'level_0': 'ID'}, inplace=True)
data = data1

# Create measures for leakage

### Clarifying fields/variables
Even with the help of the accompanying dictionary, it is good to clarify, with reasonable assumptions, what certain fields and columns mean in the context of this dataset.  

* MTRC is the unit of measure for each unique NDC. 
* As one can see that  DSPN units can be a real number with decimal points, this is reasonable.
* DSPN and PURCH are in the same unit.
* COGS_PRC_MTRC is cost **the pharmacy pays** per unit, so there is no need for conversion.
* GCN and NDC are seemingly equivalent, but data is grouped by GCN number **for each pharmacy**. Extra caution should be used when handling these fields.

### Leakage is represented by share of purchase for a type of drug
* Percentage: 
    - Percentage of (COGS of an NDC bought from AmerisourceBergen) to (COGS of dispensing units)
* If grouped by any variable, COGS needs to be aggregated before doing an average (avoiding average of averages)
* Formula: PURCH SALES / DSPN COGS ? **[Percentage will be a ratio for dollars, not quantities]**
* Columns are named "perc_?"
* (Tentative) Leakage can be represented as: 1 - Perc

In [ ]:
#Calculate and insert sales for purch
unit_cost = data['COGS_PRC_MTRC']
data.loc[:,'sales_1w'] = Series(data['PURCH_QTY_NDC_1WK_MTRC']*unit_cost, dtype='float32')
data.loc[:,'sales_4w'] = Series(data['PURCH_QTY_NDC_4WKS_MTRC']*unit_cost, dtype='float32')
data.loc[:,'sales_13w'] = Series(data['PURCH_QTY_NDC_13WKS_MTRC']*unit_cost, dtype='float32')
data.loc[:,'sales_26w'] = Series(data['PURCH_QTY_NDC_26WKS_MTRC']*unit_cost, dtype='float32')
data.loc[:,'sales_gcn_1w'] = Series(data['PURCH_QTY_GCN_1WK_MTRC']*unit_cost, dtype='float32')
data.loc[:,'sales_gcn_4w'] = Series(data['PURCH_QTY_GCN_4WKS_MTRC']*unit_cost, dtype='float32')
data.loc[:,'sales_gcn_13w'] = Series(data['PURCH_QTY_GCN_13WKS_MTRC']*unit_cost, dtype='float32')
data.loc[:,'sales_gcn_26w'] = Series(data['PURCH_QTY_GCN_26WKS_MTRC']*unit_cost, dtype='float32')
data.loc[:,'leak_dol_26w'] = Series(data['DSPN_COGS_NDC_26WKS']-data['sales_26w'])

In [ ]:
def getPerc(data):    
    data.loc[:,'perc_1w'] = Series(np.where(data['DSPN_QTY_NDC_1WK'] != 0,data['sales_1w']/data['DSPN_COGS_NDC_1WK'],0), dtype='float32')
    data.loc[:,'perc_4w'] = Series(np.where(data['DSPN_QTY_NDC_4WKS'] != 0,data['sales_4w']/data['DSPN_COGS_NDC_4WKS'],0), dtype='float32')
    data.loc[:,'perc_13w'] = Series(np.where(data['DSPN_QTY_NDC_13WKS'] != 0,data['sales_13w']/data['DSPN_COGS_NDC_13WKS'],0), dtype='float32')
    data.loc[:,'perc_26w'] = Series(np.where(data['DSPN_QTY_NDC_26WKS'] != 0,data['sales_26w']/data['DSPN_COGS_NDC_26WKS'],0), dtype='float32')
    data.loc[:,'leak_26w'] = Series(1 - data['perc_26w'])
def getPercGCN(data):
    data.loc[:,'perc_gcn_1w'] = Series(np.where(data['DSPN_QTY_GCN_1WK'] != 0,data['sales_gcn_1w']/data['DSPN_COGS_GCN_1WK'],0), dtype='float32')
    data.loc[:,'perc_gcn_4w'] = Series(np.where(data['DSPN_QTY_GCN_4WKS'] != 0,data['sales_gcn_4w']/data['DSPN_COGS_GCN_4WKS'],0), dtype='float32')
    data.loc[:,'perc_gcn_13w'] = Series(np.where(data['DSPN_QTY_GCN_13WKS'] != 0,data['sales_gcn_13w']/data['DSPN_COGS_GCN_13WKS'],0), dtype='float32')
    data.loc[:,'perc_gcn_26w'] = Series(np.where(data['DSPN_QTY_GCN_26WKS'] != 0,data['sales_gcn_26w']/data['DSPN_COGS_GCN_26WKS'],0), dtype='float32')

### Total Sales Leak

In [ ]:
data.leak_dol_26w.sum()

# Examine Pharmacies first

In [ ]:
data_bypharm = data.groupby('PHRMCY_NAME').sum().reset_index()

In [ ]:
#now you do the percentages on the dollar amount
getPerc(data_bypharm)

In [ ]:
#remove outliers
data_bypharm_excl = removeOutliers(data_bypharm,'perc_26w')

In [ ]:
#Number of unique pharmacies?
print("Number of unique pharmacies (exl. outliers) is: " + str(len(data_bypharm_excl)))

print("{:.4}% of values remain.".format(len(data_bypharm_excl)/len(data_bypharm)*100))

# Pharmacies

In [ ]:
data_bypharm.sort_values(by='leak_dol_26w', ascending=False).head(5)

* Dispensing means "sold to patients". As you can see in the table, due to existing stock, pharmacies naturally buy fewer new drugs in the short term. 
<br> _However, this means that it is difficult to use changes in purchase percentages to evaluate leakage._
* Some pharmacies bought more than they actually dispensed, but these are in the minority.

In [ ]:
leak_bypharm = data_bypharm[data_bypharm.perc_26w < 0.9]
print("{:.4}% of pharmacies dispensed more than they have bought from AB.".format(len(leak_bypharm)/len(data_bypharm)*100))
print("This means almost all pharmacies leak in some capacity")

*Consider only those less than or smaller than 1 and larger than 0 (0 suggests no relationship), as pharmacies that overbought should not be a concern*

In [ ]:
#distribution of 26-week percentage of pharmacies that leak
leak_bypharm = data_bypharm_excl[data_bypharm_excl.perc_26w <= 1]
leak_bypharm = leak_bypharm[leak_bypharm.perc_26w >= 0] 

# sb.distplot(leak_bypharm['perc_26w'], kde=True, 
#              bins=20, color = 'green',)
plt.hist(leak_bypharm['perc_26w'], bins = 20)
plt.xlabel("26-week Perc")
plt.ylabel("Number of pharmacies")
leak_bypharm['perc_26w'].describe()

## Group data by NDCs and NDC Descriptions

In [ ]:
data_byNDCdesc = data.groupby('NDC_DESC').sum().reset_index()
getPerc(data_byNDCdesc)

data_byNDC = data.groupby('NDC').sum().reset_index()
getPerc(data_byNDC)

print(data_byNDCdesc.leak_26w.describe())
print('')
print(data_byNDC.leak_26w.describe())

INSIGHT?: Percs per NDC are smaller -> There are "leaky" NDCs that are not bought even though the whole category is well-bought

In [ ]:
toplot = data_byNDCdesc[(data_byNDCdesc.perc_26w > 0) & (data_byNDCdesc.perc_26w <= 1)]
# toplot2 = data_byNDC[data_byNDC.perc_26w > 0]
# sb.distplot(toplot['perc_26w'], bins=20)
x = pd.Series(toplot['leak_26w'])
plt.figure(figsize=(7.5,2.5))
plt.hist(x, bins=np.linspace(0, 1, 21), histtype='barstacked')
plt.xticks(np.linspace(0, 1, 21), rotation=90)
plt.xlabel("26-week Leak")
plt.ylabel("Frequency of GCNs");


In [ ]:
toplot2 = data_byNDC
# sb.distplot(toplot['perc_26w'], bins=20)
y = toplot2['leak_26w']
plt.figure(figsize=(7.5,2.5))
plt.hist(y, bins=np.linspace(0, 1, 21), histtype='barstacked', color='green')
plt.xticks(np.linspace(0, 1, 21), rotation=90)
plt.xlabel("26-week Leak")
plt.ylabel("Frequency of NDCs");

In [ ]:
data_byNDCdesc.head(3)

In [ ]:
toplot = toplot.sort_values(by='leak_26w',ascending=False).reset_index()

plt.figure(figsize=(10,20))
toplot['leak_26w'].plot(kind='barh')
plt.yticks(toplot.index, toplot['NDC_DESC']);
# toplot['NDC_DESC']
# https://stackoverflow.com/questions/20548727/how-to-determine-the-order-of-bars-in-a-matplotlib-bar-chart

**Chọn mốc cắt theo ý mình**

Sau khi xóa tất cả các PURCH quantity nào mà bằng 0 -> Leak từ 0 -> 40% (Purc percentage khoàng 60-70% chỉ còn có 2 NDC)


In [6]:
gcnseq_desc = data[['GCNSEQ_DESC','NDC_DESC']].drop_duplicates('NDC_DESC')
new = data_byNDCdesc.merge(gcnseq_desc, on='NDC_DESC',how='left')
new[['NDC_DESC','leak_dol_26w','GCNSEQ_DESC']].to_csv('NDCdesc_byleakdol.csv')

NameError: name 'data' is not defined

## Examining Drug Make-up

In [ ]:
data.loc[:,'GCNSEQ_DESC'] = data['GCNSEQ_DESC'].str.replace('"','')
data.loc[:,'GCNSEQ_DESC'] = data['GCNSEQ_DESC'].str.replace('(3)','')
data.loc[:,'GCNSEQ_DESC'] = data['GCNSEQ_DESC'].str.replace('()','')

In [ ]:
dummies_desc = data['GCNSEQ_DESC'].str.get_dummies(sep=' ')

In [ ]:
dummies_desc = dummies_desc.reset_index()
dummies_desc.rename(columns={'index': 'ID'}, inplace=True);

In [ ]:
dummies_brand = data['GCNSEQ_DESC'].str.get_dummies(sep=' ')

In [ ]:
data_merged = data.merge(dummies_desc,on='ID', how='left');

### Leakage with dummy variables 

In [ ]:
data_count = data_merged.iloc[:,5:193]
data_count['cogs_26w'] = data_merged['DSPN_COGS_NDC_26WKS']
data_count['sales_26w'] = data_merged['sales_26w']
data_count['price'] = data_merged['COGS_PRC_MTRC']

In [ ]:
data_count.to_csv('data_spss.csv')

In [ ]:
getPerc(data_count)

In [ ]:
data_count

In [ ]:
data_count['cogs_26w'];

In [ ]:
d = {}
for i in data_count:
    if i not in ['cogs_26w','sales_26w','price']:
        cogs = data_count[data_count[i] == 1]['cogs_26w'].sum()
        sales = data_count[data_count[i] == 1]['sales_26w'].sum()
        d[i] = 1 - sales/cogs

In [ ]:
leak_by_ingr = pd.DataFrame.from_dict(d, orient='index',columns=['leak_26w']).reset_index()

In [ ]:
leak_by_ingr.sort_values(by='index', ascending=False).to_csv('leak_by_ingr.csv')

In [ ]:
# leak_by_ingr.head(10)

In [ ]:
k = {}
for i in data_count:
    if i not in ['cogs_26w','sales_26w','price']:
        k[i] = data_count[data_count[i] == 1]['price'].mean()

In [ ]:
price_by_ingr = pd.DataFrame.from_dict(k, orient='index',columns=['av_price']).reset_index()

In [ ]:
price_by_ingr.sort_values(by='index', ascending=False).to_csv('leak_by_price.csv')

In [ ]:
h = {}
for i in data_count:
    if i not in ['cogs_26w','sales_26w','price']:
        h[i] = data_count[data_count[i] == 1]['cogs_26w'].sum()

In [ ]:
cogs_by_ingr = pd.DataFrame.from_dict(h, orient='index',columns=['cogs_26w']).reset_index()

In [ ]:
cogs_by_ingr.sort_values(by='index', ascending=False).to_csv('cogs_by_ingr.csv')

In [ ]:
j = {}
for i in data_count:
    if i not in ['cogs_26w','sales_26w','price']:
        j[i] = data_count[data_count[i] == 1]['sales_26w'].sum()

In [ ]:
sales_by_ingr = pd.DataFrame.from_dict(j, orient='index',columns=['sales_26w']).reset_index()

In [ ]:
sales_by_ingr.sort_values(by='index', ascending=False).to_csv('sales_by_ingr.csv')

In [7]:
leak_by_ingr = leak_by_ingr.merge(price_by_ingr, on='index',how='left')
leak_by_ingr = leak_by_ingr.merge(cogs_by_ingr, on='index',how='left')
leak_by_ingr = leak_by_ingr.merge(sales_by_ingr, on='index',how='left')

NameError: name 'leak_by_ingr' is not defined

In [ ]:
leak_by_ingr.sort_values(by='index', ascending=False).to_csv('leak_ingr_summary.csv')

## _Three top leakers are different NDCs with a shared ingredients_

In [ ]:
getPerc(data_merged)
# data_merged = data_merged[data_merged.perc_26w < 1]

In [ ]:
data_merged[data_merged['emtricitab/rilpiviri/tenof'] == 1][['PHRMCY_NAME','NDC_DESC','GCNSEQ_DESC','perc_13w','perc_26w']].head(8)
#Maybe different NDC leads to 

In [ ]:
data_merged[data_merged['bictegrav/emtricit/tenofov'] == 1][['PHRMCY_NAME','NDC_DESC','GCNSEQ_DESC','perc_13w','perc_26w']].head(6)

In [ ]:
data_merged[data_merged['alafen'] == 1][['PHRMCY_NAME','NDC_DESC','GCNSEQ_DESC','perc_13w','perc_26w']].head(7)

A story here. Tenofovi Alafenamide is a new HIV drug, but seems to be marketed under different NDC (names), with slightly different mixture of ingredients.
* Perhaps the differences confuse pharmacies
* For DESCOVY and GENVOYA, pharmacies seem to underbuy (in comparision to dispense) in the long-term, then in a shorter timeframe tries to close this gap. Trust or more consumer may have helped reduce leakage.  

# Searching for patterns


## 1. Price per pharmacy, Purchase Percentage, and Manufacturer

**Hypothesis:** if an NDC is more expensive than the category average, they should exhibit higher leakage  

In [ ]:
#initial dataframe to determine aver.cost per NDC
data_mean_cost_per_NDC = data[['NDC','COGS_PRC_MTRC']].groupby('NDC').mean().reset_index()
#prep by re-introducing the 'MFG_NAM' column.
mfg_name = data[['MFG_NAM','NDC']].drop_duplicates('NDC')
#prep by re-introducing the 'NDC_DESC' column.
ndc_desc = data[['NDC_DESC','NDC']].drop_duplicates('NDC')
#merge 
scat_data = data_mean_cost_per_NDC.merge(mfg_name,on='NDC', how='left')
scat_data = scat_data.merge(ndc_desc,on='NDC', how='left')


In [ ]:
#prep by calculating a "price index" from -1 to 1, representing deviation from the average price of the GNC
gnc_num = data[['GCNSEQ_NBR','NDC']].drop_duplicates('NDC')
gnc_av_price = data[['GCNSEQ_NBR','NDC','COGS_PRC_MTRC']].groupby(['GCNSEQ_NBR']).mean().reset_index()
gnc_av_price = gnc_av_price.rename(index=str, columns={"COGS_PRC_MTRC": "average_price"})

In [ ]:
#prep by branded and generics
gen = data[['BRX_GRX_CD','NDC']].drop_duplicates('NDC')

In [ ]:
#prep by buying groups
buy = data[['NDC','BUY_GRP_NAM']].drop_duplicates('NDC')

In [ ]:
#joining columns 
scat_data = scat_data.merge(gnc_num,on='NDC', how='left')
scat_data = scat_data.merge(gnc_av_price,on='GCNSEQ_NBR', how='left')
scat_data = scat_data.merge(data_byNDC[['NDC','leak_26w']],on='NDC',how='left')
scat_data = scat_data.merge(data_byNDC[['NDC','leak_dol_26w']],on='NDC',how='left')
scat_data = scat_data.merge(gen,on='NDC', how='left')
scat_data = scat_data.merge(buy,on='NDC', how='left')

In [ ]:
scat_data.loc[:,'price_index'] = Series((scat_data.COGS_PRC_MTRC - scat_data.average_price)/scat_data.average_price)


In [ ]:
check = scat_data[['NDC','COGS_PRC_MTRC','BRX_GRX_CD','average_price','price_index','leak_26w']]
check[check.price_index > 1].head(4)
#exorbitant price compared to that of other NDCs in the same GCN

In [ ]:
#Advanced: use the actual GCN price -> Later
# gnc_av_price = data[['GCNSEQ_NBR','NDC','PHRMCY_NAME','COGS_PRC_MTRC']].groupby(['PHRMCY_NAME','GCNSEQ_NBR'])

In [ ]:
#try to limit it between 0 and 1 (0 to 100%)
# scat_data = scat_data[(scat_data.leak_26w > 0 )]
scat_data = scat_data[scat_data.price_index < 0.75]

In [ ]:
removeOutliers(scat_data,'price_index').sort_values(by='leak_26w').head(20);
scat_data = scat_data[scat_data.leak_26w < 0.8];

### Unit Price and Leakage
Based on the hypothesize that pharmacies are less loyal when it comes to costlier drug 

In [ ]:
plt.figure(figsize=(9,9))
g = sb.scatterplot(x='leak_26w', y='price_index', data=scat_data,hue="BRX_GRX_CD",size='leak_dol_26w', sizes = (150,500),legend=False)
# plt.setp(g.get_legend().get_texts(), fontsize='12')
plt.xlabel("Leakage Rate")
plt.ylabel("Price Index")
plt.savefig("price_index.png", transparent=True)

No relationship

In [ ]:
data_grx_scat = scat_data[scat_data['BRX_GRX_CD'] == 'GRX']
data_brx_scat = scat_data[scat_data['BRX_GRX_CD'] == 'BRX']

In [ ]:
plt.figure(figsize=(7,7))
l = sb.scatterplot(x='leak_26w', y='price_index', data=data_grx_scat, size='leak_dol_26w', sizes = (70,400))
# plt.setp(g.get_legend().get_texts(), fontsize='12')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel("Leakage Rate")
plt.ylabel("Price Index")
# , size='leak_dol_26w'

In [ ]:
plt.figure(figsize=(7,7))
l = sb.scatterplot(x='leak_26w', y='price_index', data=data_brx_scat, size='leak_dol_26w', sizes = (70,400))
# plt.setp(g.get_legend().get_texts(), fontsize='12')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel("Leakage Rate")
plt.ylabel("Price Index")
# , size='leak_dol_26w'

## 2. Branded and Generic NDCs

### With Price at NDC level

In [ ]:
# def removeOutliers(data,field):
#     result = data[np.abs(data[field]-data[field].mean()) <= (3*data[field].std())]
#     return result
brand = data[['NDC','BRX_GRX_CD']].drop_duplicates('NDC')
buy_group = data[['NDC','BUY_GRP_NAM']].drop_duplicates('NDC')
data_brand = data_byNDC.merge(brand,on='NDC', how='left')
data_brand = data_brand.merge(buy_group,on='NDC', how='left')

In [ ]:
data_brand = removeOutliers(data_brand,'leak_dol_26w')
data_brand = data_brand[data_brand['leak_26w'] > 0]

In [ ]:
# data_brand = data_brand[data_brand['leak_26w'] < 0.9]
data_brand = data_brand[data_brand['COGS_PRC_MTRC'] < 60000]

In [8]:
branded = data_brand[data_brand.BRX_GRX_CD == 'BRX']
generic = data_brand[data_brand.BRX_GRX_CD == 'GRX']

x = branded['perc_26w']
y = generic['perc_26w']

NameError: name 'data_brand' is not defined


> There is statistical difference after removing the values above 90%. T-test can confirm.

In [ ]:
ttest_ind(x,y)

In [ ]:
plt.figure(figsize=(6,6)) 
# sb.pairplot(x_vars=['COGS_PRC_MTRC'], y_vars=['leak_26w'], data=data_brand,hue="BRX_GRX_CD", height=6,   plot_kws=dict(s=75))
g = sb.scatterplot(x='leak_26w', y='DSPN_COGS_NDC_26WKS', data=data_brand,hue="BRX_GRX_CD", size='leak_dol_26w', sizes = (50,425))
plt.xlabel("Leakage Rate")
plt.ylabel("Dollars of drugs dispensed per NDC")


In [ ]:
x = data_count[['leak_dol_26w','leak_26w','NDC_DESC']][data_count['elviteg/cob/emtri/tenof']==1].sort_values(by='leak_dol_26w', ascending=False)
x.head(10)

### _High-spending, branded items generally leak more and cost more in their leakage._
### _Generic items may leak at a higher percentage, but do not represent much loss._ 

In [ ]:
# plt.figure(figsize=(6,6))
# sb.pairplot(x_vars=['leak_26w'], y_vars=['leak_dol_26w'], data=data_brand,hue="BRX_GRX_CD", height=6,   plot_kws=dict(s=75))
# h = sb.scatterplot(x='leak_26w', y='COGS_PRC_MTRC', data=data_brand,hue="BRX_GRX_CD", size='leak_dol_26w', sizes = (50,450))

In [ ]:
data_brx = data_brand[data_brand['BRX_GRX_CD'] == 'BRX']
data_grx = data_brand[data_brand['BRX_GRX_CD'] == 'GRX']

In [ ]:
# plt.figure(figsize=(10,10))
# # sb.pairplot(x_vars=['leak_26w'], y_vars=['leak_dol_26w'], data=data_brand,hue="BRX_GRX_CD", height=6,   plot_kws=dict(s=75))
# # m = sb.scatterplot(x='leak_26w', y='leak_dol_26w', data=data_brand,hue="BRX_GRX_CD", size='leak_dol_26w', sizes = (50,450))
# m = sb.regplot(x="leak_26w", y="COGS_PRC_MTRC", data=data_brx)

* Branded medicine suprisingly does not have high leakage rate, but represents a higher dollar value that can be captured.

Lower-priced generics have lower perc -> Higher leakage

Quartiles -> To make sure that the big ones stay in one chart

### 3. Buying Groups

In [ ]:
plt.figure(figsize=(10,10))
b = sb.scatterplot(x='leak_26w', y='COGS_PRC_MTRC', data=data_brand,hue="BUY_GRP_NAM", size='leak_dol_26w', sizes = (50,450)) 
plt.setp(b.get_legend().get_texts(), fontsize='12')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
data_buygroup = data.groupby('BUY_GRP_NAM').sum().reset_index()
getPerc(data_buygroup)

## Use this to get back to [the start](#first-bullet)
<a name='down'></a>

In [ ]:
# data_buy_price = data[data['GCNSEQ_DESC'].str.contains('insulin|linagliptin|glargine,hum.rec.anlog|sitagliptin|dulaglutide|liraglutide')]
#get data on only CLIPPERS and WARRIERS, the two jugglenaughts
# data_2 = data[data['BUY_GRP_NAM'].str.contains('CLIPPERS|LAKERS')]
data_2 = data[data['BUY_GRP_NAM'].str.contains('WARRIERS|CLIPPERS')]

#Get price info on LANTUS SOLOSTAR
data_lan_solo = data_2[data_2['NDC_DESC'].str.contains('LANTUS 100 UNIT/ML VIAL')]
# data_lan_solo = data_lan_solo[data_lan_solo['DSPN_QTY_NDC_26WKS'] <5000]
data_lan_solo = data_lan_solo[data_lan_solo['COGS_PRC_MTRC'] >25.25]
data_lan_solo = data_lan_solo[data_lan_solo['leak_dol_26w'] >0]
data_lan_solo.loc[:,'leak_26w'] = data_lan_solo['leak_dol_26w']/data_lan_solo['DSPN_COGS_NDC_26WKS']

In [ ]:
data_lan_solo[['DSPN_COGS_NDC_26WKS','sales_26w','leak_dol_26w','leak_26w']].head(10);

In [ ]:
plt.figure(figsize=(7,7))
a = sb.scatterplot(x='leak_26w', y='COGS_PRC_MTRC', data=data_lan_solo,hue="BUY_GRP_NAM", size='leak_dol_26w', sizes=(75,320),alpha=0.9,legend=False)
# plt.setp(b.get_legend().get_texts(), fontsize='12')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('Leak Rate')
plt.ylabel('Metric Price')
plt.savefig('lantus_solo.png', transparent=True)

In [ ]:
#Get price info on LANTUS 100
data_lan = data_2[data_2['NDC_DESC'].str.contains('LANTUS 100')]


In [ ]:
data_lan.loc[:,'leak_26w'] = data_lan.loc[:,'leak_dol_26w']/data_lan.loc[:,'DSPN_COGS_NDC_26WKS'];

In [ ]:
data_lan = data_lan[data_lan['leak_dol_26w'] >0]
data_lan = data_lan[data_lan['COGS_PRC_MTRC'] >25.35]

In [ ]:
data_lan[['DSPN_COGS_NDC_26WKS','sales_26w','leak_dol_26w']];

In [ ]:
plt.figure(figsize=(7,7))
k = sb.scatterplot(x='leak_26w', y='COGS_PRC_MTRC', data=data_lan,hue="BUY_GRP_NAM", size='leak_dol_26w', sizes=(75,320),alpha=0.9)
# plt.setp(b.get_legend().get_texts(), fontsize='12')\
plt.xlabel('Leak')
plt.ylabel('Metric Price')

In [ ]:
#Get price info on LEVEMIR FLEXTOUCH
data_levemir = data_2[data_2['NDC_DESC'].str.contains('NOVOLOG 100 UNITS/ML FLEXPEN')]
data_levemir = data_levemir[data_levemir['COGS_PRC_MTRC'] >35]
data_levemir = data_levemir[data_levemir['leak_dol_26w'] >0]
data_levemir.loc[:,'leak_26w'] = data_levemir['leak_dol_26w']/data_levemir['DSPN_COGS_NDC_26WKS']
data_levemir.sort_values(by='BUY_GRP_NAM', ascending=False);

In [ ]:
plt.figure(figsize=(7,7))
k = sb.scatterplot(x='leak_26w', y='COGS_PRC_MTRC', data=data_levemir,hue="BUY_GRP_NAM", size='leak_dol_26w', sizes=(75,320),alpha=0.9, legend=False)
plt.xlabel('Leak Rate')
plt.ylabel('Metric Price')
plt.savefig('lantus_novolog.png', transparent=True)

In [ ]:
#Get price info on JANUVIA
data_janu = data_2[data_2['NDC_DESC'].str.contains('JANUVIA')]
data_janu = data_janu[data_janu['COGS_PRC_MTRC'] >13]
data_janu = data_janu[data_janu['leak_dol_26w'] >0]
data_janu.loc[:,'leak_26w'] = data_janu['leak_dol_26w']/data_janu['DSPN_COGS_NDC_26WKS']

In [ ]:
plt.figure(figsize=(7,7))
k = sb.scatterplot(x='leak_26w', y='COGS_PRC_MTRC', data=data_janu,hue="BUY_GRP_NAM", size='leak_dol_26w', sizes=(65,320))
# plt.setp(b.get_legend().get_texts(), fontsize='12')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('Dispensed Total Value')
plt.ylabel('Metric Price')

In [9]:
#Get price info on TRADJENTA
data_trad = data_2[data_2['NDC_DESC'].str.contains('TRADJENTA')]
data_trad = data_trad[data_trad['COGS_PRC_MTRC'] >12.75]
data_trad = data_trad[data_trad['leak_dol_26w'] >0]
data_trad.loc[:,'leak_26w'] = data_trad['leak_dol_26w']/data_trad['DSPN_COGS_NDC_26WKS']

NameError: name 'data_2' is not defined

In [ ]:
plt.figure(figsize=(7,7))
k = sb.scatterplot(x='leak_26w', y='COGS_PRC_MTRC', data=data_trad,hue="BUY_GRP_NAM", size='leak_dol_26w', sizes=(75,320),alpha=0.9)
# plt.setp(b.get_legend().get_texts(), fontsize='12')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('Dispensed Total Value')
plt.ylabel('Metric Price')

In [ ]:
#Get price info on VICTOZA 2-PAK 
data_vic = data_2[data_2['NDC_DESC'].str.contains('VICTOZA 3-PAK')]
# data_vic = data_vic[data_vic['COGS_PRC_MTRC'] >12.75]
data_vic = data_vic[data_vic['leak_dol_26w'] >0]
data_vic = data_vic[data_vic['leak_dol_26w'] <25000]
data_vic = data_vic[data_vic['COGS_PRC_MTRC'] > 91]
data_vic.loc[:,'leak_26w'] = data_vic['leak_dol_26w']/data_vic['DSPN_COGS_NDC_26WKS']

In [ ]:
plt.figure(figsize=(7,7))
k = sb.scatterplot(x='leak_26w', y='COGS_PRC_MTRC', data=data_vic,hue="BUY_GRP_NAM", size='leak_dol_26w', sizes=(75,320),alpha=0.9, legend=False)
# plt.setp(b.get_legend().get_texts(), fontsize='12')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('Leak Rate')
plt.ylabel('Metric Price')
plt.savefig('lantus_victoza.png', transparent=True)

## What's with SPURS

## A unifying predictive model


<b>Tentative features:</b>
* Key ingredients (combine ingredients that always go together)
* Buying group (make them dummy variables too)
* DSPN dollars
* A measure of price (which accounts for actual, reasonable units)
* Branded / Generic

<b>Target:</b>
* Leak Dol


<em><b>Try Ridge Regression first, then support vector</b></em>